In [49]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder



In [19]:
train_imgs_path = '../data/fungi-clef-2025/images/FungiTastic-FewShot/train/300p/'
val_imgs_path = '../data/fungi-clef-2025/images/FungiTastic-FewShot/val/300p/'
# test_imgs_path = '../data/fungi-clef-2025/images/FungiTastic-FewShot/test/300p/'

train_metadata_path = '../data/fungi-clef-2025/metadata/FungiTastic-FewShot/FungiTastic-FewShot-Train.csv'
val_metadata_path = '../data/fungi-clef-2025/metadata/FungiTastic-FewShot/FungiTastic-FewShot-Val.csv'
# test_metadata_path = '../data/fungi-clef-2025/metadata/FungiTastic-FewShot/FungiTastic-FewShot-Test.csv'

df_train = pd.read_csv(train_metadata_path)
df_train['filepath'] = train_imgs_path + df_train['filename']

df_val = pd.read_csv(val_metadata_path)
df_val['filepath'] = val_imgs_path + df_val['filename']

In [ ]:

# enumerate class labels
le = LabelEncoder()
# Only fit on training labels
le.fit(df_train['class'])

df_train['label'] = le.transform(df_train['class'])
df_val['label'] = le.transform(df_val['class'])


In [51]:
df_train.head(3)

,eventDate,year,month,day,habitat,countryCode,scientificName,kingdom,phylum,class,...,district,filename,category_id,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion,filepath,label
0,2021-01-24,2021,1.0,24.0,Mixed woodland (with coniferous and deciduous ...,DK,Xylohypha ferruginosa (Corda) S.Hughes,Fungi,Ascomycota,Eurotiomycetes,...,Næstved,0-3052832307.JPG,2421,wood,0,0.0,16.0,continental,../data/fungi-clef-2025/images/FungiTastic-Few...,11
1,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Gribskov,0-3061954303.JPG,386,wood,0,0.0,17.0,continental,../data/fungi-clef-2025/images/FungiTastic-Few...,21
2,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Gribskov,1-3061954303.JPG,386,wood,0,0.0,17.0,continental,../data/fungi-clef-2025/images/FungiTastic-Few...,21


In [61]:
X_train = []
y_train = []

for i, row in df_train.iterrows():
    # Load image using designated filepath
    img = load_img(row['filepath'])
    img_array = img_to_array(img)

    # Need to convert to tensor to use resize_with_pad
    img_tensor = tf.convert_to_tensor(img_array)
    # Preserves aspect ratio, pads with black bars to account for resizing
    # prevents the image from getting cropped out
    img_tensor = tf.image.resize_with_pad(img_tensor, target_height=300, target_width=300)
    # print(type(img_tensor))
    img_tensor = img_tensor / 255.0  # Normalize

    X_train.append(img_tensor.numpy())  # Convert back to NumPy
    y_train.append(row['label'])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [62]:
X_val = []
y_val = []

for i, row in df_val.iterrows():
    # Load image using designated filepath
    img = load_img(row['filepath'])
    img_array = img_to_array(img)

    # Need to convert to tensor to use resize_with_pad
    img_tensor = tf.convert_to_tensor(img_array)
    # Preserves aspect ratio, pads with black bars to account for resizing
    # prevents the image from getting cropped out
    img_tensor = tf.image.resize_with_pad(img_tensor, target_height=300, target_width=300)
    # print(type(img_tensor))
    img_tensor = img_tensor / 255.0  # Normalize

    X_val.append(img_tensor.numpy())  # Convert back to NumPy
    y_val.append(row['label'])

X_val = np.array(X_val)
y_val = np.array(y_val)